In [ ]:
# DISTANCE FUNCTION
#
# INPUT: points p and q, represented as lists of coordinates
#
# OUTPUT: the Hamming distance between p and q, or the number of coordinates which differ between them
def D(p, q):
    
    d = len(p) # dimension
    
    if len(q) != d: # p and q are of different dimensions
        return None
    
    dist = 0 # to track Hamming distance
    for i in range(d): 
        if p[i] != q[i]: # p and q different at coordinate i
            dist += 1 
            
    return dist

In [ ]:
# HASH FUNCTION
#
# INPUT: a point p, represented as a tuple of coordinates; a multi-set I, represented as a list
#
# OUTPUT: the output of the function g defined by I, represented as a tuple
def g(p, I):
    
    g = []
    for i in I:
        g.append(p[i]) # where the coordinate of p at index i gives h_i(p) in the context of the Hamming metric
        
    return tuple(g)

In [ ]:
import random

# PREPROCESSING FUNCTION
#
# INPUT: a list P of d-dimension tuples representing the point set; parameters k and L denoting level of sensitivity
#      and accuracy, respectively
#
# OUTPUT: a dictionary, buckets, which stores a pointer to each p in P in the buckets g_1(p),...,g_L(p);
#      a list, G, storing the multi-sets I_1,...,I_L, represented as lists, denoting the respective functions
#      g_1,...,g_L
def preProcess(P, k, L):
    
    d = len(P[0]) # determine dimension (also the size of H in the context of the Hamming metric)
    
    buckets = {} # stores a pointer to each p in P in the buckets g_1(p),...,g_L(p)
    G = [] # stores the multi-sets I_1,...,I_L denoting the respective functions g_1,...,g_L
    
    for j in range(L): # choose L functions g_1,...,g_L from G "independently and uniformly at random"
        
        I_j = [] 
        
        # build I_j (the multi-set that determines g_j, a function which concatenates several hash functions from H)
        for i in range(k): # choose k hash functions h_i_1,...h_i_k from H
            I_j.append(random.randrange(d)) # pick indices i_1,...i_k at random
            
        G.append(I_j) # store multi-set in G for querying purposes

        # push each point p to g_j(p) bucket
        for p in P:
            
            g_j_p = g(p, I_j) # determine g_j(p)
            
            if g_j_p in buckets:
                buckets[g_j_p].add(p) # add p to existing bucket
            else:
                buckets[g_j_p] = {p} # create new bucket containing p
            
    return buckets, G

In [ ]:
# QUERY FUNCTION
#
# INPUT: a point q, represented as a tuple; radius r2; a dictionary, buckets, which stores a pointer to each p in P 
#      in the buckets g_1(p),...,g_L(p), unless p has been deleted; a list, G, storing the multi-sets I_1,...,I_L, 
#      represented as lists, denoting the respective functions g_1,...,g_L
#
# OUTPUT: a point, p, such that D(p,q) <= r2 OR None, in which case we do not guarantee that such a point does not
#      exist
def query(q, r, buckets, G):
    
    L = len(G) # level of accuracy, i.e., the number of concatenated hash functions which map to each p in P
    
    count = 0 # keeps track of number of points examined
    
    # iterate through each bucket g_1(q),...,g_L(q)
    for I_j in G:
        g_j_q = g(q, I_j)

        # carry out a brute-force search for a near-neighbor of q in the bucket if it exists
        #      (defined as a point p such that D(p,q) <= r2)
        if g_j_q in buckets: # Note: this should evaluate to True unless q is not in P

            for p in buckets[g_j_q]:
                
                count += 1

                # if there is any point p such that D(p,q) <= r2 then we return the point, else we return None 
                if D(p, q) <= r:
                    return p

                # "as it is possible that the total number of points stored in those buckets is large, we 
                #      interrupt the search after examining the first 3L points (including duplicates)""
                if count >= 3*L:
                    return None # no point p with D(p,q) <= r2 has been found
    
    return None # no point p with D(p,q) <= r2 has been found

In [ ]:
# DELETE FUNCTION
#
# INPUT: a vertex to be deleted, p, represented as a tuple; a dictionary, buckets, which stores a pointer to each
#      p in P in the buckets g_1(p),...,g_L(p) unless p has been already deleted; a list, G, storing the multi-sets 
#      I_1,...,I_L, represented as lists, denoting the respective functions g_1,...,g_L
#
# OUTPUT: no return value; the buckets dictionary will be updated such that no pointer to p is stored in any bucket
def delete(p, buckets, G):

    for I_j in G:
        g_j_p = g(p, I_j)         # for each bucket g_1(p),...,g_L(p)
        buckets[g_j_p].discard(p)  # remove p from the bucket 

In [ ]:
# EXACT CONNECTED COMPONENTS FUNCTION - used for testing against ApproximateCC
#
# INPUT: a list P of n d-dimensional tuples representing the point set; radius r
#
# OUTPUT: a partitioning, "partition", of the points in P, represented as a list of sets of tuples; a list, E_prime,
#      of the edges, represented as tuples, which form the connected components of the partitioning; two points p, q 
#      are said to be "connected" if D(p, q) <= r
def BruteForceCC(P, r):
    
    P = P.copy()

    partition = [] # the partitioning
    E_prime = []   # the edge list

    while len(P) != 0: # there are still points to input into the partitioning
    
        p = next(iter(P)) # select any p in P
        P.remove(p)       # delete p from P
        
        S = {p}   # the set of points to be queried
        C = set() # the next connected component to be computed

        # repeat until the connected component C has been completed
        while len(S) != 0:
      
            q = S.pop() # select and remove any q from S
            C.add(q)    # add q to C

            # check all points in P
            for p_prime in P:
                if D(q, p_prime) <= r:  # if "connected"
                    E_prime.append((q, p_prime)) # add the edge (q, p') to E'
                    P.remove(p_prime)            # delete p' from P
                    S.add(p_prime)               # add p' to S

        partition.append(C)  # add C to the partition
  
    return partition, E_prime

In [ ]:
import math

# APPROXIMATE CONNECTED COMPONENTS FUNCTION
#
# INPUT: a list P of n d-dimension tuples representing the point set; approximation factor c; radius r
#
# OUTPUT: a partitioning, "partition", of the points in P, represented as a list of sets of tuples; a list, E_prime,
#      of the edges, represented as tuples, which form the connected components of the partitioning; two points p, q 
#      are said to be "connected" if D(p, q) <= r. The returned partitioning is a c-approximation of the exact
#      connected components, i.e., two points p, q may be in the same component if D(p, q) <= c*r
def ApproximateCC(P, c, r):
    
    P = P.copy()

    # By Proposition 3.8:
    # set p1 = 1 - r/d
    # set p2 = 1 - rc/d
    # The definition of the family H of hash functions is thus (r, rc, p1, p2)-sensitive
    
    d = len(P[0]) # dimension of each point
    
    p1 = 1 - (r / d)
    p2 = 1 - (r*c / d)
    
    # By Proof of Theorem 3.4:
    # set k = ceil(log(n, 1/p2))
    # set L = n^rho/p1
    # where rho = log(1/p1) / log(1/p2)
    # The above gives us a failure probability (that of the query function falsely returning no neighbor)
    #      of at most f = 1/3 + 1/e < 1for the LSH construction of the near-neighbor data structure

    n = len(P) # the size of the point set

    rho = math.log(1 / p1) / math.log(1 / p2)

    k = math.ceil(math.log(n, 1 / p2))
    L = int(n**rho / p1)
    
    r2 = c*r # used by the query function to report an approximate near-neighbor

    # construct a near-neighbor data structure for P
    buckets, G = preProcess(P, k, L)

    partition = [] # the partitioning
    E_prime = []   # the edge list

    while len(P) != 0: # there are still points to input into the partitioning
    
        p = next(iter(P))  # select any p in P

        # delete p from P and from the associated near-neighbor structure
        P.remove(p)
        delete(p, buckets, G)  

        S = {p}   # the set of points to be queried
        C = set() # the next connected component to be computed

        # repeat until S = ø, i.e., connected component C has been completed
        while len(S) != 0:
            
            q = S.pop() # select and remove any q from S
            C.add(q)    # add q to C

            # let p' be the answer to the query of the near-neighbor structure on q
            p_prime = query(q, r2, buckets, G)

            # repeat until p' is null
            while p_prime is not None:

                E_prime.append((q,p_prime))  # add the edge (q, p') to E'

                # delete p' from P and the associated NearNbr structure
                P.remove(p_prime)
                delete(p_prime, buckets, G)

                S.add(p_prime)  # add p' to S

                p_prime = query(q, r2, buckets, G)  # choose new p'

        partition.append(C)  # add C to the partitioning

    return partition, E_prime

In [ ]:
# PRIM'S ALGORITHM
#
# INPUT: a list P of n d-dimensional tuples representing the point set
#
# OUTPUT: E, a dictionary matching each point p in P to the edge connecting p to its parent in the MST; edge_sum,
#      a sum over all the weights of the edges in E
def Prim(P):
    
    P = P.copy() # create copy so as to not overwrite P
    
    d = len(P[0]) # the dimension of each point in P
    
    C = [] # weights of the shortest edge from each remaining point in P to the tree
    E = {} # shortest edge from each point to the tree, or the edge from each point to its parent once it has been
           #      inserted into the tree
    
    for i in range(len(P)):
        C.append(d + 1) # d is the greatest possible distance between two points
        E[P[i]] = False # no edges have been examined yet
            
    edge_sum = 0

    while (len(P) != 0): # while there are still points to examine
        
        i = C.index(min(C)) # select the point which has the shortest edge to the tree
        wt = C.pop(i)
        
        if wt < d + 1: 
            edge_sum += wt # add the weight of the edge to the sum
        
        v = P.pop(i) # v is the newly added point to the tree
                
        for j in range(len(P)): # iterate over all remaining points
            w = P[j]
            
            # if the edge between v and w is shorter than the shortest edge connecting w to other points in the tree
            if D(v, w) < C[j]:
                C[j] = D(v, w) # update the edge weight
                E[w] = (v, w)  # update the shortest edge
            
    return E, edge_sum

In [ ]:
# APPROXIMATE MST ALGORITHM
#
# INPUT: a list P of n d-dimensional tuples representing the point set; approximation factors c, gamma
#
# OUTPUT: components, a dictionary mapping each point to its respective component in the partitioning as the tree 
#      is built, represented as a set; partitioning, a list of sets of points representing the connected components 
#      in the graph; E, a set of edges making up the tree; edge_sum, a sum over the weights of all edges in E
def ApproximateMST(P, c, gamma):
    
    P = P.copy() # create copy so as to not overwrite P
    
    E = set() # the edge set
    
    d = len(P[0]) # the dimension of each point
    r = d         # the maximum value of D(p, q) for any two points p, q is d
            
    n = len(P) # the number of points
    M = int(math.log(n / gamma, 1 + gamma)) # the number of radii to be considered
    
    R = [] # list of radii
    for i in range(M + 1): # for i = 0,...,M
        r_i = r / math.pow(1 + gamma, M - i)
        if r_i*c >= d: # instead of padding points with dummy zeros to increase dimension, we ensure here that p2 > 0
            R.append((d - 1) / c)
            break
        elif r_i*c >= 1:
            R.append(r_i) # for the Hamming metric only, r < 1 is pointless to consider with distinct points
            
    components = dict() # partitioning dictionary which maps each point to its respective component
    partitioning = []   # list of the components in the partitioning
    edge_sum = 0        # the sum over all edges in the tree
    
    for p in P: # initially, each point has its own component in the partiitoning
        components[p] = {p}
        partitioning.append({p})
                
    for i in range(len(R)): # for i = 0,...,M do
        partitioning_i, E_prime = ApproximateCC(P, c, R[i]) # invoke ApproximateCC(P,c,r_i)

        for e in E_prime: # for each edge (u, v) in E' do
            
            u, v = e
            
            P_i = components[u] 
            P_j = components[v]
                                
            if P_i != P_j: # if u and v belong to different sets P_i, P_j in the partitioning then
                
                partitioning.remove(P_i)
                partitioning.remove(P_j)
                
                P_new = P_i.union(P_j)
                
                # merge P_i and P_j in the partitioning
                for p in P_new:
                    components[p] = P_new
                partitioning.append(P_new)
                
                E.add(e) # add e to the edge set
                edge_sum += D(u, v)  # add the edge sum

    if len(partitioning) != 1: # in the case that the graph is not connected at the end of the algorithm
        
        P_random = []
        
        for comp in partitioning: # select a random point from each component of the partitioning
            P_random.append(random.choice(list(comp)))
            
        addl_E, addl_sum = Prim(P_random) # run Prim's algorithm on the random points
        
        for p in P_random:
            
            if addl_E[p]: # i.e., not the root
                
                u, v = addl_E[p]
            
                P_i = components[u] 
                P_j = components[v]

                if P_i != P_j: # if u and v belong to different sets P_i, P_j in the partitioning then

                    partitioning.remove(P_i)
                    partitioning.remove(P_j)

                    P_new = P_i.union(P_j)

                    # merge P_i and P_j in the partitioning
                    for p in P_new:
                        components[p] = P_new
                    partitioning.append(P_new)

                    E.add((u, v)) # add (u, v) to the edge set
                    edge_sum += D(u, v)  # add the edge sum
        
    return components, partitioning, E, edge_sum

In [ ]:
# TREE CHECKER FUNCTION
#
# INPUT: n, the number of points in a graph; partitioning, a list of sets of points representing the connected
#      components of the graph; E, a list representing the edge set of the graph
#
# OUTPUT: True, if the provided graph is a tree, and False otherwise; relies on graph theory that a connected graph
#      with n-1 edges is acyclic and hence is a tree
def isTree(n, partitioning, E):
    
    if len(partitioning) != 1: # graph is not connected (too many components)
        print("fail a")
        return False
        
    if len(partitioning[0]) != n: # graph is not connected (missing vertices)
        print("fail b")
        return False
    
    if len(E) != n-1: # graph is either not connected (too few edges) or is cyclic (too many edges)
        print("fail c")
        return False
    
    for e in E:
        (u, v) = e
        if (v, u) in E: # graph is not connected (duplicate edges in set leading to too few edges)
            print("fail d")
            return False
        
    return True

In [ ]:
# RANDOM POINT SET GENERATION FUNCTION #1: "UNCLUSTERED"
#
# INPUT: sigma, a list representation of the alphabet from which to choose point coordinates; d, the dimension of 
#     each point; n, the number of points to be generated
#
# OUTPUT: a list P of n d-dimensional tuples, with each coordinate of each point chosen independently and uniformly
#      at random from sigma, with the exception of the first coordinate, which is equal to the index of the point in
#      P to ensure points are distinct (i.e., have a Hamming distance >= 1)
def unclusteredPointSet(sigma, d, n):
    
    P = [] # the list of points
    
    for i in range(n): # n points to be generated
        
        p = [] # list representation of the point
        
        p.append(i) # ensure points are distinct (i.e., Hamming metric >= 1) by setting first coordinate equal to i
        
        for j in range(d - 1): # points are d-dimensional
            p.append(random.choice(sigma)) # each coordinate chosen independently and uniformly at random from sigma
            
        P.append(tuple(p)) # cast p as a tuple and add it to P
    
    return P

In [ ]:
# RANDOM POINT SET GENERATION FUNCTION #2: "PROBABILITY CLUSTERS"
#
# INPUT: d, the dimension of each point; n, the number of points to be generated
#
# OUTPUT: a list P of n d-dimensional tuples, each in one of 11 clusters—coordinates of points in the 0,...,10th  
#      cluster are chosen independently at random with a 0,...,10/10 probability of being 0 and 10,...,0/10 
#      probability of being 1, respectively, with the exception of the first coordinate, which is equal to the index
#      of the point in P to ensure points are distinct (i.e., have a Hamming distance >= 1)
def probClustersPointSet(d, n):
    
    P = [] # the list of points
    
    for i in range(n): # n points to be generated
        
        cluster = i // (n // 11) # cluster index representing the probability (out of 10) of choosing a 0 over a 1
        
        p = [] # list representation of the point
        
        p.append(i) # ensure points are distinct (i.e., Hamming metric >= 1) by setting first coordinate equal to i
        
        for j in range(d - 1): # points are d-dimensional
            
            pick = random.choice(range(11)) 
            if pick < cluster: 
                p.append(0) # select 0 with cluster/10 probability
            else:
                p.append(1) # select 1 with 1 - cluster/10 probability
    
        P.append(tuple(p)) # cast p as a tuple and add it to P

    return P

In [ ]:
# RANDOM POINT SET GENERATION FUNCTION #3: "MATCHED CLUSTERS"
#
# INPUT: d, the dimension of each point; n, the number of points to be generated
#
# OUTPUT: a list P of n d-dimensional tuples, each in one of len(sigma) clusters—the first 20% of coordinates of each
#      point are chosen independently and uniformly at random from sigma, with the exception of the first coordinate,
#      which is equal to the index of the point in P to ensure points are distinct (i.e., have a Hamming distance 
#      >= 1); the remaining 80% of coordinates correspond to the element of sigma associated with the cluster;
#      points within a cluster have a Hamming distance of at most d/5 and points in different clusters have a Hamming
#      distance of at least 4d/5
def matchedClustersPointSet(sigma, d, n):
    
    P = [] # the list of points
    
    for i in range(n): # n points to be generated
        
        k = i // (n // len(sigma)) # cluster index representing the element from sigma making up 80% of coordinates
        
        p = [] # list representation of the point
        
        p.append(i) # ensure points are distinct (i.e., Hamming metric >= 1) by setting first coordinate equal to i
        
        for j in range(1, d): # points are d-dimensional 
            
            if j < d / 5: # first 20% of coordinates chosen independently and uniformly at random from sigma
                p.append(random.choice(sigma))
                
            else: # remaining 80% of coordinates correspond to the element of sigma associated with the cluster
                p.append(sigma[k-1])
                
        P.append(tuple(p)) # cast p as a tuple and add it to P
            
    return P

In [ ]:
# LOAD PLANTS TEST DATA
import csv

csv_filename = '/Users/elizacrocker/Desktop/Thesis/data/plants_data.csv' # replace with correct pathname

with open(csv_filename) as file:
    reader = csv.reader(file)
    next(reader) # skip header line
    plants = list(tuple(line) for line in reader)

In [ ]:
import time

# TESTING PLANTS DATA

c = [10, 20, 50, 100]
gamma = 1

for i in range(len(c)):
    for j in range(5): # 5 trials for each c value
        a_time = time.time()
        comp, part, E_approx, approx_edge_sum = ApproximateMST(plants, c[i], gamma)
        a_time = time.time() - a_time

        print(isTree(len(plants), part, E_approx))
        print(len(plants), "c =", c[i], "approx:", approx_edge_sum, a_time)

In [ ]:
## TEST ENVIRONMENT

import time
import matplotlib.pyplot as plt

theoretical = []

approx_times = []
prim_times = []

approx_sums = []
exact_sums = []

d = 100       # dimension
gamma = 1     # approximation factor
sigma = [0,1] # alphabet
#n = 500

for n_div in range(1, 11):
    n = 10*n_div
    
    approx_times_2 = []
    approx_sums_2 = []
    
    approx_times_3 = []
    approx_sums_3 = []
    
    approx_times_4 = []
    approx_sums_4 = []
    
    approx_times_10 = []
    approx_sums_10 = []
    
    prim_times = []
    exact_sums = []
    
    for i in range(5): # repeat each c value 5 times

        # choose point set generation function
#         P = unclusteredPointSet(sigma, d, n)
#         P = probClustersPointSet(d, n)
        P = matchedClustersPointSet(sigma, d, n)
    
        ##### TEST C = 2 #####
        a_time = time.time()
        comp, part, E_approx, approx_edge_sum = ApproximateMST(P, 2, gamma)
        a_time = time.time() - a_time

        approx_sums_2.append(approx_edge_sum)
        approx_times_2.append(a_time)

        if not isTree(n, part, E_approx):
            print(n, c)
            print(len(P))
            print(len(part))
            print(len(E_approx))
           
        ##### TEST C = 3 #####
        a_time = time.time()
        comp, part, E_approx, approx_edge_sum = ApproximateMST(P, 3, gamma)
        a_time = time.time() - a_time

        approx_sums_3.append(approx_edge_sum)
        approx_times_3.append(a_time)

        if not isTree(n, part, E_approx):
            print(n, c)
            print(len(P))
            print(len(part))
            print(len(E_approx))
            
        ##### TEST C = 4 #####
        a_time = time.time()
        comp, part, E_approx, approx_edge_sum = ApproximateMST(P, 4, gamma)
        a_time = time.time() - a_time

        approx_sums_4.append(approx_edge_sum)
        approx_times_4.append(a_time)

        if not isTree(n, part, E_approx):
            print(n, c)
            print(len(P))
            print(len(part))
            print(len(E_approx))
 
        ##### TEST C = 10 #####
        a_time = time.time()
        comp, part, E_approx, approx_edge_sum = ApproximateMST(P, 10, gamma)
        a_time = time.time() - a_time

        approx_sums_10.append(approx_edge_sum)
        approx_times_10.append(a_time)

        if not isTree(n, part, E_approx):
            print(n, c)
            print(len(P))
            print(len(part))
            print(len(E_approx))
              
        ##### TEST PRIM #####
        p_time = time.time()
        E_prim, prim_edge_sum = Prim(P)
        p_time = time.time() - p_time

        exact_sums.append(prim_edge_sum)
        prim_times.append(p_time)
        
    print(n)
    print("approx sums, c = 2")
    for s in approx_sums_2:
        print(s)

    print("approx times, c = 2")
    for s in approx_times_2:
        print(s)
        
    print("approx sums, c = 3")
    for s in approx_sums_3:
        print(s)

    print("approx times, c = 3")
    for s in approx_times_3:
        print(s)
        
    print("approx sums, c = 4")
    for s in approx_sums_4:
        print(s)

    print("approx times, c = 4")
    for s in approx_times_4:
        print(s)

    print("approx sums, c = 10")
    for s in approx_sums_10:
        print(s)

    print("approx times, c = 10")
    for s in approx_times_10:
        print(s)

    print("exact sums")
    for s in exact_sums:
        print(s)

    print("prim times")
    for s in prim_times:
        print(s)
        
    print()

In [ ]:
### THIS CODE TESTED TAKING IN THE EXISTING PARTITIONING AND ADDING CONNECTED COMPONENTS TO THE SET OF QUERY POINTS IMMEDIATELY

import math

def ApproximateCC2(P, partitions, c, r):

    # By Proposition 3.8:
    # set p1 = 1 - r/d
    # set p2 = 1 - rc/d
    d = len(P[0])
    p1 = 1 - (r / d)
    p2 = 1 - (r*c / d)


    # By Proof of Theorem 3.4:
    # set k = ceil(log(n, 1/p2))
    # set L = n^rho/p1
    # where rho = log(1/p1) / log(1/p2)

    n = len(P)

    arg1 = 1 / p1
    arg2 = 1 / p2

    rho = math.log(arg1) / math.log(arg2)

    k = math.ceil(math.log(n, arg2))
    L = int(n**rho / p1)
    
    r2 = c*r

    # construct a NearNbr(P,c,r,f) data structure for P
    # NOTE: the LSH construction of the NearNbr data structure gives us a
    # failure probability of f <= 1/3 + 1/e < 1
    buckets, G = Hash(P, k, L)
    
    partition = []
    E_prime = []

    while len(P) != 0:
    
        p = next(iter(P))  # select any p in P

        # delete p and already connected components from P and from the associated NearNbr structure
        S = {p}
        P.remove(p)
        delete(p, buckets, G)
                
        for el in partitions[p]:
            if el in P:
                S.add(el)
                P.remove(el)
                delete(el, buckets, G)
        C = set() # the next connected component to be computed

        # repeat until S = ø, i.e., connected component C has been completed
        while len(S) != 0:

            q = S.pop()     # select and remove any q from S
            C.add(q)        # add q to C

            # let p' be the answer of the NearNbr structure on q
            p_prime = query(q, r2, buckets, G)

            # repeat until p' is null
            while p_prime is not None:

                E_prime.append((q,p_prime))  # add {q, p'} to E'

                # delete p' from P and the associated NearNbr structure
                P.remove(p_prime)
                delete(p_prime, buckets, G)

                S.add(p_prime)  # add p' to S
                
                for el in partitions[p_prime]:
                    if el in P:
                        S.add(el)
                        P.remove(el)
                        delete(el, buckets, G)

                p_prime = query(q, r2, buckets, G)  # choose new p'

        partition.append(C)  # add C to the partition

    return partition, E_prime

In [ ]:
### THIS CODE TESTED CHOOSING A POINT FROM EACH COMPONENT OF THE EXISTING PARTITIONING AT RANDOM TO QUERY

import math

def ApproximateCC3(P, p_list, p_map, c, r):

    # By Proposition 3.8:
    # set p1 = 1 - r/d
    # set p2 = 1 - rc/d
    d = len(P[0])
    p1 = 1 - (r / d)
    p2 = 1 - (r*c / d)


    # By Proof of Theorem 3.4:
    # set k = ceil(log(n, 1/p2))
    # set L = n^rho/p1
    # where rho = log(1/p1) / log(1/p2)

    n = len(P)

    arg1 = 1 / p1
    arg2 = 1 / p2

    rho = math.log(arg1) / math.log(arg2)

    k = math.ceil(math.log(n, arg2))
    L = int(n**rho / p1)
    
    r2 = c*r

    # construct a NearNbr(P,c,r,f) data structure for P
    # NOTE: the LSH construction of the NearNbr data structure gives us a
    # failure probability of f <= 1/3 + 1/e < 1
    buckets, G = Hash(P, k, L)

    partition = []
    E_prime = []

    while len(p_list) != 0:
    
        part = next(iter(p_list))  # select any p in P
        p = random.choice(list(part))

        # delete p and already connected components from P and from the associated NearNbr structure
        S = {p}
                
        for el in part:
            delete(el, buckets, G)
                
        p_list.remove(part)
                
        C = set() # the next connected component to be computed

        # repeat until S = ø, i.e., connected component C has been completed
        while len(S) != 0:
            
            q = S.pop()     # select and remove any q from S
            C.update(p_map[q])        # add q to C

            # let p' be the answer of the NearNbr structure on q
            p_prime = query(q, r2, buckets, G)

            # repeat until p' is null
            while p_prime is not None:

                E_prime.append((q,p_prime))  # add {q, p'} to E'

                S.add(p_prime)  # add p' to S
                
                P_p_prime = p_map[p_prime]
                
                for el in P_p_prime:
                    delete(el, buckets, G)
                        
                p_list.remove(P_p_prime)
                
                p_prime = query(q, r2, buckets, G)  # choose new p'

        partition.append(C)  # add C to the partition

    return partition, E_prime

In [ ]:
### THIS CODE SUPPORTED ApproximateCC2

import time

def ApproximateMST2(P, c, gamma):
    
    E = set()
    q = random.choice(P)
    
    d = len(P[0])
    r = d
            
    n = len(P)
    M = int(math.log(n / gamma, 1 + gamma))
    
    R = []
    for i in range(M + 1):
        r_i = r / math.pow(1 + gamma, M - i)
        if r_i * c >= d:
            R.append((d - 1) / c)
            break
        elif r_i >= 1: # only for Hamming
            R.append(r_i)
        
    partitions = dict()
    edge_sum = 0
    
    for p in P:
        partitions[p] = {p}
                
    for i in range(len(R)):
        part, E_prime = ApproximateCC2(P.copy(), partitions, c, R[i])
        for e in E_prime:
            
            u, v = e
            wt = D(u, v)
                        
            P_i = partitions[u]
            P_j = partitions[v]
                                
            if P_i != P_j:
                P_new = P_i.union(P_j)
                for p in P_new:
                    partitions[p] = P_new
                E.add(e)
                edge_sum += wt
    return edge_sum, E, partitions

In [ ]:
### THIS CODE SUPPORTED ApproximateCC3

import time

def ApproximateMST3(P, c, gamma):
    
    E = set()
    
    d = len(P[0])
    r = d
            
    n = len(P)
    M = int(math.log(n / gamma, 1 + gamma))
    
    R = []
    for i in range(M + 1):
        r_i = r / math.pow(1 + gamma, M - i)
        if r_i * c >= d:
            R.append((d - 1) / c)
            break
        elif r_i >= 1: # only for Hamming
            R.append(r_i)
    R.append((d - 1) / c)

        
    partitions_list = []
    partitions_map = dict()
    edge_sum = 0
    
    for p in P:
        partitions_list.append({p})
        partitions_map[p] = partitions_list[-1]
                
    for i in range(len(R)):
        part, E_prime = ApproximateCC3(P.copy(), partitions_list.copy(), partitions_map.copy(), c, R[i])

        for e in E_prime:
            
            u, v = e
            wt = D(u, v)
                    
            P_i = partitions_map[u]
            P_j = partitions_map[v]

                
            if P_i != P_j:
                new_part = P_i.union(P_j)
                partitions_list.remove(P_i)
                partitions_list.remove(P_j)
                partitions_list.append(new_part)
                for q in new_part:
                    partitions_map[q] = new_part
                E.add(e)
                edge_sum += wt
     
    return edge_sum, E, partitions_list